In [18]:
import os
import gc
import time
import re
import ast
import country_converter as coco
import time
import requests
import json
import googlemaps
import pandas as pd
from urllib.parse import quote_plus
from collections import Counter
from functools import lru_cache
from tqdm import tqdm

# Manage API Keys
from dotenv import load_dotenv
load_dotenv()
user_agent_NOMINATIM = os.getenv("USER_AGENT_NOMINATIM")

import logging
coco_logger = logging.getLogger('country_converter.country_converter')
coco_logger.setLevel(logging.ERROR)

In [2]:
DF_isna_input = input().strip()

 D:\Data_40_years_cancer_studies\BERT_NER_parsedXMLs_temp_isna_notna_base\


In [7]:
df = pd.read_csv(DF_isna_input + "BERT_NER_temp_isna_notna.csv")

In [11]:
# Path to the .csv file with geographical entities information 
entities_database_input = input().strip()

 C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\resources\allCountries_clean.csv


In [12]:
# Path to the .csv file with scientific institutions information
institutions_database_input = input().strip()

 C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\resources\ScimagoIR 2025 - Overall Rank.csv


In [13]:
# Import databases
entities_database = pd.read_csv(entities_database_input)
institutions_database = pd.read_csv(institutions_database_input, sep=";")

# Remove " *" from institutions when present
institutions_database["Institution"] = institutions_database["Institution"].apply(lambda x: x[:-2] if x[-2:] == " *" else x)

In [14]:
df_fixed = df.loc[df["Country"].notna()]
df_to_fix = df.loc[df["Country"].isna()]

In [15]:
df_to_fix[["PMID", "Authors", "NER_lastAuthor", "Country", "Country_source"]]

,PMID,Authors,NER_lastAuthor,Country,Country_source
0,28520365,"[{'Name': 'Laura Dean', 'Affiliation': ['NCBI'...",[],NaN,NaN
1,28520372,"[{'Name': 'Laura Dean', 'Affiliation': ['NCBI'...",[],NaN,NaN
2,28520376,"[{'Name': 'Laura Dean', 'Affiliation': ['NCBI'...",[],NaN,NaN
3,28520377,"[{'Name': 'Laura Dean', 'Affiliation': ['NCBI'...",[],NaN,NaN
4,28723017,"[{'Name': 'Ari D. Leib', 'Affiliation': ['Aden...","[{'text': 'Farber Cancer Institute', 'label': ...",NaN,NaN
...,...,...,...,...,...
141696,11265390,"[{'Name': 'G Namysłowski', 'Affiliation': ['II...",[],NaN,NaN
141697,11265404,"[{'Name': 'J Takahashi', 'Affiliation': ['Dept...","[{'text': 'Radiation Oncology', 'label': 'ORG'}]",NaN,NaN
141698,11265406,"[{'Name': 'H Terashima', 'Affiliation': ['Dept...","[{'text': 'Surgery, Hiraka General Hospital', ...",NaN,NaN
141699,11265412,"[{'Name': 'M Ishizaki', 'Affiliation': ['Dept....",[],NaN,NaN


In [16]:
df_to_fix[["PMID", "Authors", "NER_BERT", "Country", "Country_source"]]

,PMID,Authors,NER_BERT,Country,Country_source
0,28520365,"[{'Name': 'Laura Dean', 'Affiliation': ['NCBI'...","[{'entity_group': 'INS', 'score': np.float32(0...",NaN,NaN
1,28520372,"[{'Name': 'Laura Dean', 'Affiliation': ['NCBI'...","[{'entity_group': 'INS', 'score': np.float32(0...",NaN,NaN
2,28520376,"[{'Name': 'Laura Dean', 'Affiliation': ['NCBI'...","[{'entity_group': 'INS', 'score': np.float32(0...",NaN,NaN
3,28520377,"[{'Name': 'Laura Dean', 'Affiliation': ['NCBI'...","[{'entity_group': 'INS', 'score': np.float32(0...",NaN,NaN
4,28723017,"[{'Name': 'Ari D. Leib', 'Affiliation': ['Aden...","[{'entity_group': 'INS', 'score': np.float32(0...",NaN,NaN
...,...,...,...,...,...
141696,11265390,"[{'Name': 'G Namysłowski', 'Affiliation': ['II...","[{'entity_group': 'INS', 'score': np.float32(0...",NaN,NaN
141697,11265404,"[{'Name': 'J Takahashi', 'Affiliation': ['Dept...","[{'entity_group': 'INS', 'score': np.float32(0...",NaN,NaN
141698,11265406,"[{'Name': 'H Terashima', 'Affiliation': ['Dept...","[{'entity_group': 'INS', 'score': np.float32(0...",NaN,NaN
141699,11265412,"[{'Name': 'M Ishizaki', 'Affiliation': ['Dept....","[{'entity_group': 'INS', 'score': np.float32(0...",NaN,NaN


In [17]:
# Define a dictionary for US states. 
# Many article's affiliation have the state name/ code and not "United States"
us_states = [
    "Alabama", "AL", "Al",
    "Alaska", "AK", "Ak",
    "Arizona", "AZ", "Az",
    "Arkansas", "AR", "Ar",
    "California", "CA", "Ca",
    "Colorado", "CO", "Co",
    "Connecticut", "CT", "Ct",
    "Delaware", "DE", "De",
    "Florida", "FL", "Fl",
    "Georgia", "GA", "Ga",
    "Hawaii", "HI", "Hi",
    "Idaho", "ID", "Id",
    "Illinois", "IL", "Il",
    "Indiana", "IN", "In",
    "Iowa", "IA", "Ia",
    "Kansas", "KS", "Ks",
    "Kentucky", "KY", "Ky",
    "Louisiana", "LA", "La",
    "Maine", "ME", "Me",
    "Maryland", "MD", "Md",
    "Massachusetts", "MA", "Ma",
    "Michigan", "MI", "Mi",
    "Minnesota", "MN", "Mn",
    "Mississippi", "MS", "Ms",
    "Missouri", "MO", "Mo",
    "Montana", "MT", "Mt",
    "Nebraska", "NE", "Ne",
    "Nevada", "NV", "Nv",
    "New Hampshire", "NH", "Nh",
    "New Jersey", "NJ", "Nj",
    "New Mexico", "NM", "Nm",
    "New York", "NY", "Ny",
    "North Carolina", "NC", "Nc",
    "North Dakota", "ND", "Nd",
    "Ohio", "OH", "Oh",
    "Oklahoma", "OK", "Ok",
    "Oregon", "OR", "Or",
    "Pennsylvania", "PA", "Pa",
    "Rhode Island", "RI", "Ri",
    "South Carolina", "SC", "Sc",
    "South Dakota", "SD", "Sd",
    "Tennessee", "TN", "Tn",
    "Texas", "TX", "Tx",
    "Utah", "UT", "Ut",
    "Vermont", "VT", "Vt",
    "Virginia", "VA", "Va",
    "Washington", "WA", "Wa",
    "West Virginia", "WV", "Wv",
    "Wisconsin", "WI", "Wi",
    "Wyoming", "WY", "Wy",
    "District of Columbia", "DC", "Dc"
]

In [20]:
# Precompute lookup tables
institution_to_country = dict(zip(institutions_database["Institution"], institutions_database["Country"]))
entity_to_country = dict(zip(entities_database["name"], entities_database["country code"]))
all_institutions_set = set(institution_to_country.keys())

# Instance conerter
cc = coco.CountryConverter()

# MUST include contact info. Example format: 'Application_name/1.0 (yourEmail@provider.com)'
headers = {
    'User-Agent': user_agent_NOMINATIM  
}

# Compiled regex list if regex matching is required
compiled_institution_patterns = [(inst, re.compile(re.escape(inst))) for inst in institution_to_country]

@lru_cache(maxsize=1000)
def get_country_from_text(text):
    """Fast country name conversion with cache."""
    try:
        # Attempt to extract country directly from text passed to the function (NER)
        result = cc.convert(names=text, to='name_short')
        return result if result != "not found" else None
    except:
        return None

def extract_country_with_states(cell_content):
    # If NER extracted nothing, return None
    if pd.isna(cell_content):
        return None, None

    try:
        # Try to convert the content in the NER field to list
        ## First, clean it from np function names
        cell_content = re.sub(r'np\.float32\(([^)]+)\)', r'\1', cell_content)
        
        parsed_content = ast.literal_eval(cell_content)
        # Make a separate variable for the organizations
        parsed_content_ins = [ins for ins in parsed_content if ins["entity_group"] == "INS"]
    except:
        return None, None

    # If output is not a list, return None
    if not isinstance(parsed_content, list):
        return None, None

    countries = []
    text_blob = []

    # NER field should contain several dictionaries (one per entity found by NER)
    for el in parsed_content:
        if isinstance(el, dict):
            # Separate text and label (INS, SUBC, COU, etc.)
            text = el.get("word", "").strip(".") # Remove points as trailing spaces after/before texts
            label = el.get("entity_group", "")

            # At some point passing the whole string to an API might be needed, so they are prepared
            text_blob.append(text)

            # Check if any named entity corresponds to a US state. If so, return United States
            if text in us_states:
                return "United States", "State_in_US"

            # For LOC and GPE labels (i.e. locations and geopolitical entities), if one entity is a country, store it
            if label in {"COU"}:
                country = get_country_from_text(text)
                if isinstance(country, str):
                    countries.append(country)

    # If only one country found, return it
    if len(countries) == 1:
        return countries[0], "Direct_country"
    # If several countries found, return the most common
    elif len(countries) > 1:
        # Return most common
        try:
            return Counter(countries).most_common(1)[0][0], "Most_common_list_countries"
        except:
            with open(DF_input + "logs errors Most common list countries.txt", "a") as f:
                f.write(str(cell_content) + ", " + str(countries) + "\n")
                
        
    # No matches from COU, try INS matching to scientific institutions DF
    entity_text = " ".join(text_blob)
    for el in reversed(parsed_content):
        if el.get("entity_group") == "INS":
            name = el["word"].strip(".")

            # Direct match
            if name in institution_to_country:
                return get_country_from_text(institution_to_country[name]), "Direct_institution"

            # Fallback regex search
            matches = [inst for inst, pattern in compiled_institution_patterns if pattern.search(name)]
            if len(matches) == 1:
                return get_country_from_text(institution_to_country[matches[0]]), "Regex_institution"

    # Try entity database
    for el in reversed(parsed_content):
        name = el.get("word", "").strip(".")
        if name in entity_to_country:
            country_code = entity_to_country[name]
            country = get_country_from_text(country_code)
            if country and re.search(r'\b' + re.escape(country) + r'\b', entity_text):
                return country, "Entity_database"

    # If all else fails, try calls to location APIs
    # Some calls to Nominatim will be done by passing all text collected by NER. This needs to be prepared
    # Merge all texts into one string, in case needed for passing to code below
    string_parts = [el["word"] for el in parsed_content if isinstance(el, dict) and "word" in el]
    string = ", ".join(string_parts) + ", " if string_parts else ""
    all_text = string[:-2] # Remove last " ,"
    
    # Prepare for submitting to Nominatim
    all_text_submit = quote_plus(all_text)
    
    
    # Other calls will use just the last ORG-labelled text (corresponding to the Institution). This needs to be prepared
    if len(parsed_content_ins) > 0:
        last_org = parsed_content_ins[-1]
    else:
        last_org = ""

    if isinstance(last_org, str):
        last_org_submit = quote_plus(last_org)
    else:
        last_org_submit = ""
    
    # Call Nominatim with the institution only (last ORG colledted by NER)
    result = requests.get("https://nominatim.openstreetmap.org/search?q="+last_org_submit+"&format=json&addressdetails=1", headers=headers)
    if result.status_code == 200:
        content = json.loads(result.content)
    
        # If Nominatim retrieves only one item, return its corresponding country
        if len(content) == 1:
            candidate = content[0]["address"]["country_code"]
            return get_country_from_text(candidate), "Nominatim"
    
        # Otherwise, try with the whole affiliation in Nominatim
        else:
            time.sleep(1)
            result_all = requests.get(
                "https://nominatim.openstreetmap.org/search?q=" + all_text_submit + "&format=json&addressdetails=1", 
                headers=headers
            )
            try:
                if result_all.content:
                    content_all = result_all.json()
                else:
                    content_all = []
            except json.JSONDecodeError:
                content_all = []
    
            # Again, if Nominatim retrieves only one item, return its corresponding country
            if len(content_all) == 1:
                if "address" in content_all[0].keys() and "country_code" in content_all[0]["address"]:
                    candidate = content_all[0]["address"]["country_code"]
                    return get_country_from_text(candidate), "Nominatim"
       
    return None, None

In [35]:
# Break dataset in 4 parts and iterate over them to get the Country data to avoid losing all information if failure happens
list_df_to_fix = []

list_df_to_fix.append(df_to_fix.iloc[:31000])
list_df_to_fix.append(df_to_fix.iloc[31000:62000])
list_df_to_fix.append(df_to_fix.iloc[62000:93000])
list_df_to_fix.append(df_to_fix.iloc[93000:])

In [44]:
start = time.time()
tqdm.pandas()
print(f"Parsing csv to fix")
for el in [1,2,3]:
    temp_df = list_df_to_fix[el]
    temp_df[["Country", "Country_source"]] = temp_df["NER_BERT"].progress_apply(extract_country_with_states).apply(pd.Series)
    temp_df.to_csv(DF_isna_input + 'fixed_with_NER_BERT_' + str(el) +'.csv', index=False)
print(f"--Parsing time: {str(round(time.time()-start, 2))}")
print("Parsing finished")

Parsing csv to fix


100%|█████████████████████████████████████████████████████████████████████████| 31000/31000 [16:44:26<00:00,  1.94s/it]
C:\Users\svalb\AppData\Local\Temp\ipykernel_4356\3258416539.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[["Country", "Country_source"]] = temp_df["NER_BERT"].progress_apply(extract_country_with_states).apply(pd.Series)
100%|█████████████████████████████████████████████████████████████████████████| 31000/31000 [13:01:18<00:00,  1.51s/it]
C:\Users\svalb\AppData\Local\Temp\ipykernel_4356\3258416539.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

--Parsing time: 151346.5
Parsing finished


In [66]:
len(df_to_fix.loc[df_to_fix['Country'].isna()])

121561

In [82]:
df_to_fix.iloc[6]["NER_BERT"]

"[{'entity_group': 'INS', 'score': np.float32(0.9998581), 'word': 'Wayne State University', 'start': 0, 'end': 22}, {'entity_group': 'INS', 'score': np.float32(0.9997116), 'word': 'Presbyterian Healthcare Services', 'start': 54, 'end': 86}, {'entity_group': 'INS', 'score': np.float32(0.9994793), 'word': 'Ascension Hospital', 'start': 88, 'end': 106}, {'entity_group': 'INS', 'score': np.float32(0.999459), 'word': 'Sichuan University', 'start': 139, 'end': 157}, {'entity_group': 'COU', 'score': np.float32(0.9989292), 'word': 'China', 'start': 159, 'end': 164}]"

In [83]:
cell_content = df_to_fix.iloc[6]["NER_BERT"]
cell_content = re.sub(r'np\.float32\(([^)]+)\)', r'\1', cell_content)
        
parsed_content = ast.literal_eval(cell_content)
# Make a separate variable for the organizations
parsed_content_ins = [ins for ins in parsed_content if ins["entity_group"] == "INS"]

parsed_content

[{'entity_group': 'INS',
  'score': 0.9998581,
  'word': 'Wayne State University',
  'start': 0,
  'end': 22},
 {'entity_group': 'INS',
  'score': 0.9997116,
  'word': 'Presbyterian Healthcare Services',
  'start': 54,
  'end': 86},
 {'entity_group': 'INS',
  'score': 0.9994793,
  'word': 'Ascension Hospital',
  'start': 88,
  'end': 106},
 {'entity_group': 'INS',
  'score': 0.999459,
  'word': 'Sichuan University',
  'start': 139,
  'end': 157},
 {'entity_group': 'COU',
  'score': 0.9989292,
  'word': 'China',
  'start': 159,
  'end': 164}]

In [84]:
parsed_content_ins

[{'entity_group': 'INS',
  'score': 0.9998581,
  'word': 'Wayne State University',
  'start': 0,
  'end': 22},
 {'entity_group': 'INS',
  'score': 0.9997116,
  'word': 'Presbyterian Healthcare Services',
  'start': 54,
  'end': 86},
 {'entity_group': 'INS',
  'score': 0.9994793,
  'word': 'Ascension Hospital',
  'start': 88,
  'end': 106},
 {'entity_group': 'INS',
  'score': 0.999459,
  'word': 'Sichuan University',
  'start': 139,
  'end': 157}]